In [1]:
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate
from keras.optimizers import adam_v2,gradient_descent_v2
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
import pandas as pd
import glob
import os
from time import time
import multiprocess as mp
import tensorflow as tf
from multiprocessing import Pool
from keras.layers import GRU,Bidirectional,LSTM,MaxPool1D,GlobalAveragePooling1D
from keras.callbacks import EarlyStopping
import eval_util as eval
import mean_average_precision_calculator as map_calculator
import average_precision_calculator


# Metrics

In [ ]:
#Metrics 
def ap_at_n(data):
    # based on https://github.com/google/youtube-8m/blob/master/average_precision_calculator.py
    predictions, actuals = data
    n = 20
    total_num_positives = None

    if len(predictions) != len(actuals):
        raise ValueError("the shape of predictions and actuals does not match.")

    if n is not None:
        if not isinstance(n, int) or n <= 0:
            raise ValueError("n must be 'None' or a positive integer."
                             " It was '%s'." % n)

    ap = 0.0

    sortidx = np.argsort(predictions)[::-1]

    if total_num_positives is None:
        numpos = np.size(np.where(actuals > 0))
    else:
        numpos = total_num_positives

    if numpos == 0:
        return 0

    if n is not None:
        numpos = min(numpos, n)
    delta_recall = 1.0 / numpos
    poscount = 0.0

    # calculate the ap
    r = len(sortidx)
    if n is not None:
        r = min(r, n)
    for i in range(r):
        if actuals[sortidx[i]] > 0:
            poscount += 1
            ap += poscount / (i + 1) * delta_recall
    return ap


def gap(pred, actual):
    lst = zip(list(pred), list(actual))
    with mp.Pool() as pool:
        all = pool.map(ap_at_n, lst)

    return np.mean(all)

In [ ]:
def sample_frame(frame_data,n_frame_sample):

  rgb_by_vid = list(map(
      lambda frames: 
      np.array(frames)[
        np.random.choice(len(frames),size=n_frame_sample)
        ],frame_data))

  X= np.array(rgb_by_vid)
  return(X)

In [ ]:
FOLDER = '/Users/marlynehakizimana/Documents/SPRING/IDS705/FinalProj/frame/' #path to: train, val, test folders with *tfrecord files
def process_records(data,tp='test'):
    #print(batch)
    tfiles = sorted(glob.glob(os.path.join(FOLDER, tp, '*tfrecord')))
    # train=['train0093.tfrecord','train0111.tfrecord','train0208.tfrecord','train0274.tfrecord','train076.tfrecord']
    # validation=['validate0052.tfrecord','validate0259.tfrecord','validate0267.tfrecord']
    # test=['validate3731.tfrecord']
    #print('total files in %s %d' % (tp, len(tfiles)))
    #tfiles=[FOLDER+tp+"/"+i for i in data]
    ids,aud,rgbs, lbs = [],[],[],[]
    for fn in tfiles :
        
        for example in tf.data.TFRecordDataset(fn).take(500):#tf.python_io.tf_record_iterator(fn):
            tf_example = tf.train.SequenceExample()#tf.train.Example.FromString(example)
            rt=tf_example.ParseFromString(example.numpy())
            yss = np.array(tf_example.context.feature["labels"].int64_list.value)
            out = np.zeros(2000).astype(np.int8) #number of classes 1000
            rgb=[]
            audio=[]
            ty=len(tf_example.feature_lists.feature_list['rgb'].feature)
            #print("long",len(yss))
            if np.sum([True for i in yss if i<=1000])==len(yss) and ty>200 :
                for y in yss:
                    out[y] = 1
                for k in range(200):#np.random.randint(0,,100):
                    rgb.append(np.array(tf.io.decode_raw(tf_example.feature_lists.feature_list['rgb'].feature[k].bytes_list.value[0],tf.uint8)))
                    audio.append(np.array(tf.io.decode_raw(tf_example.feature_lists.feature_list['audio'].feature[k].bytes_list.value[0],tf.uint8)))
                ids.append(tf_example.context.feature["id"].bytes_list.value[0].decode(encoding="UTF-8"))
                lbs.append(out)
                aud.append(audio)
                rgbs.append(rgb)
    
       
    return np.array(ids),np.array(aud), np.array(rgbs), np.array(lbs)

In [ ]:
def fc_block(x,n=4096):
    x = Dense(n)(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x=GlobalAveragePooling1D(data_format='channels_last',keepdims=False)(x)
    x = Dropout(0.2)(x)
    return x
def build_mod():
    #in1 = Input((50,128), name='x1')
    #print(in1.shape)
    #x1 = fc_block(in1)
    #print(x1.shape)
    in1 = Input((50,1024), name='x1')
    x1 = fc_block(in1)
    x=x1
    #print(x2.shape)
    #x = Concatenate(axis=-1)([x1, x2])
    #print(x.shape)
    #x = Dense(1024)(x)
    #x = BatchNormalization()(x)
    #x = LeakyReLU()(x)
    #x = Dropout(0.2)(x)
    #x=fc_block(x)
    #print(x)
    out = Dense(2000, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=[in1], outputs=out)
    #print(model.summary())
    opt = adam_v2.Adam(learning_rate=0.0001)
    #opt=gradient_descent_v2.SGD(learning_rate=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy')
    return model

In [ ]:
def train(X_train,Y_train,X_val,Y_val):
    #if not os.path.exists('weights'): os.mkdir('weights')
    model = build_mod()
    callback = EarlyStopping(monitor='loss', patience=3)
    model.fit(X_train[1],Y_train,epochs=100,batch_size=50,
                 validation_data = (X_val[1],Y_val),callbacks=[callback])
    model.save_weights('weights.h5')
    model.load_weights('weights.h5')
    y_prd = model.predict({'x1': X_val[1]}, verbose=False, batch_size=50)
    g = gap(y_prd, Y_val)
    hit=eval.calculate_hit_at_one(y_prd, y_val)
    print("Hit at one",hit)
    print("val GAP",g)
    

In [ ]:
def process_labels(data, tp='test'):
    #print(batch)
    #tfiles=[FOLDER+tp+"/"+i for i in data]
    #print('total files in %s %d' % (tp, len(tfiles)))
    tfiles = sorted(glob.glob(os.path.join(FOLDER, tp, '*tfrecord')))
    ids,ys=[],[]
    for fn in tfiles:
        
        for example in tf.data.TFRecordDataset(fn).take(500):
            tf_example = tf.train.SequenceExample()
            rt=tf_example.ParseFromString(example.numpy())
            
            yss = np.array(tf_example.context.feature["labels"].int64_list.value)
            ty=len(tf_example.feature_lists.feature_list['rgb'].feature)
            if np.sum([True for i in yss if i<=1000])==len(yss)and ty>200:
                ids.append(tf_example.context.feature["id"].bytes_list.value[0].decode(encoding="UTF-8"))
                ys.append(yss)
            
    return np.array(ys,dtype=object), np.array(ids)
def conv_pred(el): #gives the top 20 classes 
    t = 10
    idx = np.argsort(el)[::-1]
    return [el[i] for i in idx[:t]]#' '.join(['{} {:0.5f}'.format(i, el[i]) for i in idx[:t]])
def another_pred(el): #gives the top 20 classes 
    t = 10
    idx = np.argsort(el)[::-1]
    return [i for i in idx[:t]]
def predict(idx,X_test,Y_test,ylabels):
    model = build_mod()
    model.load_weights('weights.h5')
    ypd = model.predict({'x1': X_test[1]}, verbose=1, batch_size=10)
    g = gap(ypd, Y_test)
    hit=eval.calculate_hit_at_one(ypd, Y_test)
    print("Hit at one",hit)
    print("int",g)
    #del x1_val, x2_val
    with mp.Pool() as pool:
        out = pool.map(conv_pred, list(ypd))
        out1 = pool.map(another_pred, list(ypd))
    df = pd.DataFrame.from_dict({'VideoId': idx,"True labels":ylabels,"Pred labels":out1, 'Confidence': out})
    return df

# Data

In [ ]:
train_data=['train0093.tfrecord','train0111.tfrecord','train0208.tfrecord','train0274.tfrecord','train0276.tfrecord']
validation_data=['validate0052.tfrecord','validate0259.tfrecord','validate0267.tfrecord']
test_data=['validate3731.tfrecord']

#_, x1_val, x2_val, y_val = process_records(validation_data,'validation')
_, x1_val, x2_val, y_val = process_records('validation','validation')
print(x1_val.shape)
#_, x1_train, x2_train, y_train=process_records(train_data,'train')
_, x1_train, x2_train, y_train=process_records('train','train')
print(x1_train.shape)
#idx, x1_test, x2_test, y_test=process_records(test_data,'test')
idx, x1_test, x2_test, y_test=process_records('test','test')
print(x1_test.shape)
#ylabels,ids=process_labels(test_data,'test')
ylabels,ids=process_labels('test','test')

x1_val=sample_frame(x1_val,50)
x2_val=sample_frame(x2_val,50)
x1_train=sample_frame(x1_train,50)
x2_train=sample_frame(x2_train,50)
x1_test=sample_frame(x1_test,50)
x2_test=sample_frame(x2_test,50)
X_train,Y_train=[x1_train,x2_train],y_train
X_val,Y_val=[x1_val,x2_val],y_val
idx,X_test,Y_test,ylabels=idx,[x1_test,x2_test],y_test,ylabels

(5592, 200, 128)
(9978, 200, 128)
(2251, 200, 128)


# Pooling RGB

In [ ]:
train(X_train,Y_train,X_val,Y_val)

Epoch 1/100
50/50 [==============================] - 72s 1s/step - loss: 19.9818 - val_loss: 24.8436
Epoch 2/100
50/50 [==============================] - 76s 2s/step - loss: 14.9344 - val_loss: 17.4164
Epoch 3/100
50/50 [==============================] - 70s 1s/step - loss: 11.6524 - val_loss: 15.1138
Epoch 4/100
50/50 [==============================] - 65s 1s/step - loss: 9.9360 - val_loss: 14.2380
Epoch 5/100
50/50 [==============================] - 63s 1s/step - loss: 9.4918 - val_loss: 14.2949
Epoch 6/100
50/50 [==============================] - 65s 1s/step - loss: 9.9338 - val_loss: 15.8687
Epoch 7/100
50/50 [==============================] - 64s 1s/step - loss: 11.1191 - val_loss: 18.5803
Epoch 8/100
50/50 [==============================] - 62s 1s/step - loss: 13.0051 - val_loss: 21.4933
Hit at one 0.6045918367346939
val GAP 0.4989032024114854


In [ ]:
df_pooling_rgb=predict(idx,X_test,Y_test,ylabels)
#Hit@1=0.51
#Gap=0.42

15/15 [==============================] - 3s 161ms/step
Hit at one 0.5067567567567568
int 0.42127686131652853


In [ ]:
#save csv
#df_pooling_rgb.to_csv("data/Frame_level_Pooling_rgb.csv")

# POOLING AUDIO

In [ ]:
def build_mod():
    in1 = Input((50,128), name='x1')
    #print(in1.shape)
    x1 = fc_block(in1)
    #print(x1.shape)
    #in1 = Input((50,1024), name='x1')
    #x1 = fc_block(in1)
    x=x1
    #print(x2.shape)
    #x = Concatenate(axis=-1)([x1, x2])
    #print(x.shape)
    #x = Dense(1024)(x)
    #x = BatchNormalization()(x)
    #x = LeakyReLU()(x)
    #x = Dropout(0.2)(x)
    #x=fc_block(x)
    #print(x)
    out = Dense(2000, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=[in1], outputs=out)
    #print(model.summary())
    opt = adam_v2.Adam(learning_rate=0.0001)
    #opt=gradient_descent_v2.SGD(learning_rate=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy')
    return model
def train(X_train,Y_train,X_val,Y_val):
    #if not os.path.exists('weights'): os.mkdir('weights')
    model = build_mod()
    callback = EarlyStopping(monitor='loss', patience=3)
    model.fit(X_train[0],Y_train,epochs=100,batch_size=50,
                 validation_data = (X_val[0],Y_val),callbacks=[callback])
    model.save_weights('weights.h5')
    model.load_weights('weights.h5')
    y_prd = model.predict({'x1': X_val[0]}, verbose=False, batch_size=50)
    g = gap(y_prd, Y_val)
    hit=eval.calculate_hit_at_one(y_prd, y_val)
    print("Hit at one",hit)
    print("val GAP",g)
def predict(idx,X_test,Y_test,ylabels):
    model = build_mod()
    model.load_weights('weights.h5')
    ypd = model.predict({'x1': X_test[0]}, verbose=1, batch_size=10)
    g = gap(ypd, Y_test)
    hit=eval.calculate_hit_at_one(ypd, Y_test)
    print("Hit at one",hit)
    print("int",g)
    #del x1_val, x2_val
    with mp.Pool() as pool:
        out = pool.map(conv_pred, list(ypd))
        out1 = pool.map(another_pred, list(ypd))
    df = pd.DataFrame.from_dict({'VideoId': idx,"True labels":ylabels,"Pred labels":out1, 'Confidence': out})
    return df

In [ ]:
train(X_train,Y_train,X_val,Y_val)

Epoch 1/100
50/50 [==============================] - 46s 901ms/step - loss: 19.9022 - val_loss: 27.4925
Epoch 2/100
50/50 [==============================] - 44s 887ms/step - loss: 15.2852 - val_loss: 24.1111
Epoch 3/100
50/50 [==============================] - 45s 892ms/step - loss: 12.9610 - val_loss: 22.0214
Epoch 4/100
50/50 [==============================] - 36s 714ms/step - loss: 12.1187 - val_loss: 20.8249
Epoch 5/100
50/50 [==============================] - 41s 821ms/step - loss: 12.3000 - val_loss: 21.6275
Epoch 6/100
50/50 [==============================] - 51s 1s/step - loss: 13.2056 - val_loss: 22.1990
Epoch 7/100
50/50 [==============================] - 42s 833ms/step - loss: 14.8120 - val_loss: 24.0021
Hit at one 0.39285714285714285
val GAP 0.32824689326821616


In [ ]:
df_pooling_audio=predict(idx,X_test,Y_test,ylabels)
#Hit@1=0.358
#Gap=0.30

15/15 [==============================] - 1s 46ms/step
Hit at one 0.3581081081081081
int 0.30496295924959493


In [ ]:
#df_pooling_audio.to_csv("data/Frame_level_Pooling_audio.csv")

# POOLING ALL

In [ ]:
def build_mod():
    in1 = Input((50,128), name='x1')
    #print(in1.shape)
    x1 = fc_block(in1)
    #print(x1.shape)
    in2 = Input((50,1024), name='x2')
    x2 = fc_block(in2)
    #x=x1
    #print(x2.shape)
    x = Concatenate(axis=-1)([x1, x2])
    #print(x.shape)
    # x = Dense(1024)(x)
    # x = BatchNormalization()(x)
    # x = LeakyReLU()(x)
    # x = Dropout(0.2)(x)
    #x=fc_block(x)
    #print(x)
    out = Dense(2000, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=[in1,in2], outputs=out)
    #print(model.summary())
    #opt = adam_v2.Adam(learning_rate=0.0001)
    opt = adam_v2.Adam(learning_rate=0.00001)
    #opt=gradient_descent_v2.SGD(learning_rate=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy')
    return model
def train(X_train,Y_train,X_val,Y_val):
    #if not os.path.exists('weights'): os.mkdir('weights')
    model = build_mod()
    callback = EarlyStopping(monitor='loss', patience=3,restore_best_weights=True)
    model.fit([X_train[0],X_train[1]],Y_train,epochs=100,batch_size=50,
                 validation_data = ([X_val[0],X_val[1]],Y_val),callbacks=[callback])
    model.save_weights('weights.h5')
    model.load_weights('weights.h5')
    y_prd = model.predict({'x1': X_val[0],'x2':X_val[1]}, verbose=False, batch_size=50)
    g = gap(y_prd, Y_val)
    hit=eval.calculate_hit_at_one(y_prd, y_val)
    print("Hit at one",hit)
    print("val GAP",g)
def predict(idx,X_test,Y_test,ylabels):
    model = build_mod()
    model.load_weights('weights.h5')
    ypd = model.predict({'x1': X_test[0],'x2':X_test[1]}, verbose=1, batch_size=10)
    g = gap(ypd, Y_test)
    hit=eval.calculate_hit_at_one(ypd, Y_test)
    print("Hit at one",hit)
    print("int",g)
    #del x1_val, x2_val
    with mp.Pool() as pool:
        out = pool.map(conv_pred, list(ypd))
        out1 = pool.map(another_pred, list(ypd))
    df = pd.DataFrame.from_dict({'VideoId': idx,"True labels":ylabels,"Pred labels":out1, 'Confidence': out})
    return df

In [ ]:
train(X_train,Y_train,X_val,Y_val)

Epoch 1/100
200/200 [==============================] - 347s 2s/step - loss: 20.7740 - val_loss: 18.7114
Epoch 2/100
200/200 [==============================] - 2480s 12s/step - loss: 17.7959 - val_loss: 16.0977
Epoch 3/100
200/200 [==============================] - 426s 2s/step - loss: 15.3052 - val_loss: 14.2629
Epoch 4/100
200/200 [==============================] - 415s 2s/step - loss: 13.4969 - val_loss: 13.0455
Epoch 5/100
200/200 [==============================] - 797s 4s/step - loss: 12.3347 - val_loss: 12.2834
Epoch 6/100
200/200 [==============================] - 871s 4s/step - loss: 11.6337 - val_loss: 11.8660
Epoch 7/100
200/200 [==============================] - 828s 4s/step - loss: 11.2927 - val_loss: 11.7532
Epoch 8/100
200/200 [==============================] - 896s 4s/step - loss: 11.2286 - val_loss: 11.8730
Epoch 9/100
200/200 [==============================] - 955s 5s/step - loss: 11.3406 - val_loss: 12.2127
Epoch 10/100
200/200 [==============================] - 996s 5

In [ ]:
df_pooling_all=predict(idx,X_test,Y_test,ylabels)
#Hit@1=0.67
#Gap=0.632

226/226 [==============================] - 27s 117ms/step
Hit at one 0.6748111950244335
int 0.6322671281774411


In [ ]:
#df_pooling_all.to_csv("data/Frame_level_Pooling_all.csv")
df_pooling_all.to_pickle("data/Frame_level_Pooling_all_1.pkl")

# VIDEO-LEVEL_DENSE LAYER- AUDIO

In [ ]:
def avg_pooling(frame_data):
    # take avaerge across the frames for each video
    avg_rgb_by_vid = list(map(
        lambda frames: 
        np.array(frames).mean(axis=0),frame_data))

    X= np.array(avg_rgb_by_vid)
    return X

In [ ]:
def process_records(data,tp='test'):
    #print(batch)
    #tfiles = sorted(glob.glob(os.path.join(FOLDER, tp, '*tfrecord')))
    # train=['train0093.tfrecord','train0111.tfrecord','train0208.tfrecord','train0274.tfrecord','train076.tfrecord']
    # validation=['validate0052.tfrecord','validate0259.tfrecord','validate0267.tfrecord']
    # test=['validate3731.tfrecord']
    #print('total files in %s %d' % (tp, len(tfiles)))
    tfiles=[FOLDER+tp+"/"+i for i in data]
    ids,aud,rgbs, lbs = [],[],[],[]
    for fn in tfiles :
        
        for example in tf.data.TFRecordDataset(fn):#tf.python_io.tf_record_iterator(fn):
            tf_example = tf.train.SequenceExample()#tf.train.Example.FromString(example)
            rt=tf_example.ParseFromString(example.numpy())
            yss = np.array(tf_example.context.feature["labels"].int64_list.value)
            out = np.zeros(2000).astype(np.int8) #number of classes 1000
            rgb=[]
            audio=[]
            ty=len(tf_example.feature_lists.feature_list['rgb'].feature)
            #print("long",len(yss))
            if np.sum([True for i in yss if i<=1000])==len(yss):
                for y in yss:
                    out[y] = 1
                for k in range(ty):#np.random.randint(0,,100):
                    rgb.append(np.array(tf.io.decode_raw(tf_example.feature_lists.feature_list['rgb'].feature[k].bytes_list.value[0],tf.uint8)))
                    audio.append(np.array(tf.io.decode_raw(tf_example.feature_lists.feature_list['audio'].feature[k].bytes_list.value[0],tf.uint8)))
                ids.append(tf_example.context.feature["id"].bytes_list.value[0].decode(encoding="UTF-8"))
                lbs.append(out)

                aud.append(np.mean(np.array(audio),axis=0))
                rgbs.append(np.mean(np.array(rgb),axis=0))
    
       
    return np.array(ids),np.array(aud), np.array(rgbs), np.array(lbs)
def process_labels(data, tp='test'):
    #print(batch)
    tfiles=[FOLDER+tp+"/"+i for i in data]
    #print('total files in %s %d' % (tp, len(tfiles)))
    ids,ys=[],[]
    for fn in tfiles:
        
        for example in tf.data.TFRecordDataset(fn):
            tf_example = tf.train.SequenceExample()
            rt=tf_example.ParseFromString(example.numpy())
            
            yss = np.array(tf_example.context.feature["labels"].int64_list.value)
            ty=len(tf_example.feature_lists.feature_list['rgb'].feature)
            if np.sum([True for i in yss if i<=1000])==len(yss):
                ids.append(tf_example.context.feature["id"].bytes_list.value[0].decode(encoding="UTF-8"))
                #print(ids[-1])
                ys.append(yss)
    return np.array(ys,dtype=object), np.array(ids)

## Video-level Data Preprocessing

In [ ]:
_, x1_val, x2_val, y_val = process_records(validation_data,'validation')
print(x1_val.shape)
_, x1_train, x2_train, y_train=process_records(train_data,'train')
print(x1_train.shape)
idx, x1_test, x2_test, y_test=process_records(test_data,'test')
print(x1_test.shape)
ylabels,ids=process_labels(test_data,'test')


X_train,Y_train=[x1_train,x2_train],y_train
X_val,Y_val=[x1_val,x2_val],y_val
idx,X_test,Y_test,ylabels=idx,[x1_test,x2_test],y_test,ylabels

(647, 128)
(3886, 128)
(219, 128)


In [ ]:
def fc_block(x, n=4096, d=0.2):
    x = Dense(n)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(d)(x)
    return x
def build_mod():
    #in1 = Input((128), name='x1')
    in1 = Input((128,), name='x1')
    print(in1.shape)
    #print(in1.shape)
    x1 = fc_block(in1)
    #print(x1.shape)
    #in2 = Input((50,1024), name='x2')
    #x2 = fc_block(in2)
    x=x1
    #print(x2.shape)
    #x = Concatenate(axis=-1)([x1, x2])
    #print(x.shape)
    # x = Dense(1024)(x)
    # x = BatchNormalization()(x)
    # x = LeakyReLU()(x)
    # x = Dropout(0.2)(x)
    #x=fc_block(x)
    #print(x)
    out = Dense(2000, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=[in1], outputs=out)
    #print(model.summary())
    opt = adam_v2.Adam(learning_rate=0.000001)
    #opt=gradient_descent_v2.SGD(learning_rate=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy')
    return model
def train(X_train,Y_train,X_val,Y_val):
    #if not os.path.exists('weights'): os.mkdir('weights')
    model = build_mod()
    callback = EarlyStopping(monitor='loss', patience=3,restore_best_weights=True)
    #[X_train[0],X_train[1]]
    #[X_val[0],X_val[1]]
    model.fit(X_train[0],Y_train,epochs=100,batch_size=50,
                 validation_data = (X_val[0],Y_val),callbacks=[callback])
    model.save_weights('weights.h5')
    model.load_weights('weights.h5')
    y_prd = model.predict({'x1': X_val[0]}, verbose=False, batch_size=50)
    g = gap(y_prd, Y_val)
    hit=eval.calculate_hit_at_one(y_prd, y_val)
    print("Hit at one",hit)
    print("val GAP",g)

def predict(idx,X_test,Y_test,ylabels):
    model = build_mod()
    model.load_weights('weights.h5')
    ypd = model.predict({'x1': X_test[0]}, verbose=1, batch_size=10)
    g = gap(ypd, Y_test)
    hit=eval.calculate_hit_at_one(ypd, Y_test)
    print("Hit at one",hit)
    print("int",g)
    #del x1_val, x2_val
    with mp.Pool() as pool:
        out = pool.map(conv_pred, list(ypd))
        out1 = pool.map(another_pred, list(ypd))
    df = pd.DataFrame.from_dict({'VideoId': idx,"True labels":ylabels,"Pred labels":out1, 'Confidence': out})
    return df

In [ ]:
train(X_train,Y_train,X_val,Y_val)

(None, 128)
Epoch 1/100
78/78 [==============================] - 12s 143ms/step - loss: 21.8669 - val_loss: 24.0770
Epoch 2/100
78/78 [==============================] - 11s 142ms/step - loss: 21.5962 - val_loss: 21.2297
Epoch 3/100
78/78 [==============================] - 12s 154ms/step - loss: 21.3730 - val_loss: 20.6992
Epoch 4/100
78/78 [==============================] - 12s 151ms/step - loss: 21.1309 - val_loss: 20.4538
Epoch 5/100
78/78 [==============================] - 10s 131ms/step - loss: 20.9080 - val_loss: 20.2518
Epoch 6/100
78/78 [==============================] - 11s 135ms/step - loss: 20.6669 - val_loss: 20.0607
Epoch 7/100
78/78 [==============================] - 11s 139ms/step - loss: 20.4429 - val_loss: 19.8698
Epoch 8/100
78/78 [==============================] - 11s 137ms/step - loss: 20.2343 - val_loss: 19.6872
Epoch 9/100
78/78 [==============================] - 11s 136ms/step - loss: 20.0228 - val_loss: 19.5051
Epoch 10/100
78/78 [==============================] 

In [ ]:
df_VidDense_audio=predict(idx,X_test,Y_test,ylabels)
#Hit@1=0.46
#Gap=0.41

(None, 128)
22/22 [==============================] - 0s 10ms/step
Hit at one 0.4611872146118721
int 0.40696939756934875


In [ ]:
#df_VidDense_audio.to_csv("data/Video_level_Dense_audio.csv")

# VIDEO-LEVEL_DENSE LAYER- RGB

In [ ]:
def fc_block(x, n=4096, d=0.2):
    x = Dense(n)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(d)(x)
    return x
def build_mod():
    #in1 = Input((50,128), name='x1')
    #print(in1.shape)
    #x1 = fc_block(in1)
    #print(x1.shape)
    in2 = Input((1024,), name='x2')
    x2 = fc_block(in2)
    x=x2
    #print(x2.shape)
    #x = Concatenate(axis=-1)([x1, x2])
    #print(x.shape)
    # x = Dense(1024)(x)
    # x = BatchNormalization()(x)
    # x = LeakyReLU()(x)
    # x = Dropout(0.2)(x)
    #x=fc_block(x)
    #print(x)
    out = Dense(2000, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=[in2], outputs=out)
    #print(model.summary())
    opt = adam_v2.Adam(learning_rate=0.00001)
    #opt=gradient_descent_v2.SGD(learning_rate=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy')
    return model
def train(X_train,Y_train,X_val,Y_val):
    #if not os.path.exists('weights'): os.mkdir('weights')
    model = build_mod()
    callback = EarlyStopping(monitor='loss', patience=3,restore_best_weights=True)
    #[X_train[0],X_train[1]]
    #[X_val[0],X_val[1]]
    model.fit(X_train[1],Y_train,epochs=100,batch_size=50,
                 validation_data = (X_val[1],Y_val),callbacks=[callback])
    model.save_weights('weights.h5')
    model.load_weights('weights.h5')
    y_prd = model.predict({'x2': X_val[1]}, verbose=False, batch_size=50)
    g = gap(y_prd, Y_val)
    hit=eval.calculate_hit_at_one(y_prd, y_val)
    print("Hit at one",hit)
    print("val GAP",g)

def predict(idx,X_test,Y_test,ylabels):
    model = build_mod()
    model.load_weights('weights.h5')
    ypd = model.predict({'x2': X_test[1]}, verbose=1, batch_size=10)
    g = gap(ypd, Y_test)
    hit=eval.calculate_hit_at_one(ypd, Y_test)
    print("Hit at one",hit)
    print("int",g)
    #del x1_val, x2_val
    with mp.Pool() as pool:
        out = pool.map(conv_pred, list(ypd))
        out1 = pool.map(another_pred, list(ypd))
    df = pd.DataFrame.from_dict({'VideoId': idx,"True labels":ylabels,"Pred labels":out1, 'Confidence': out})
    return df

In [ ]:
train(X_train,Y_train,X_val,Y_val)

Epoch 1/100
78/78 [==============================] - 16s 187ms/step - loss: 20.8629 - val_loss: 23.3939
Epoch 2/100
78/78 [==============================] - 14s 180ms/step - loss: 18.5631 - val_loss: 17.1703
Epoch 3/100
78/78 [==============================] - 15s 194ms/step - loss: 16.7047 - val_loss: 15.8512
Epoch 4/100
78/78 [==============================] - 13s 168ms/step - loss: 15.1702 - val_loss: 14.9623
Epoch 5/100
78/78 [==============================] - 12s 158ms/step - loss: 13.9246 - val_loss: 14.2324
Epoch 6/100
78/78 [==============================] - 13s 161ms/step - loss: 12.9156 - val_loss: 13.6287
Epoch 7/100
78/78 [==============================] - 14s 178ms/step - loss: 12.1720 - val_loss: 13.1803
Epoch 8/100
78/78 [==============================] - 14s 179ms/step - loss: 11.5716 - val_loss: 12.8429
Epoch 9/100
78/78 [==============================] - 12s 159ms/step - loss: 11.0958 - val_loss: 12.6066
Epoch 10/100
78/78 [==============================] - 13s 168ms/

In [ ]:
df_Dense_rgb=predict(idx,X_test,Y_test,ylabels)
#Hit@1=0.56
#Gap=0.48

22/22 [==============================] - 0s 14ms/step
Hit at one 0.5616438356164384
int 0.47673294853661496


In [ ]:
#df_Dense_rgb.to_csv("data/Video_level_Dense_rgb.csv")

# Video_level-Dense-ALL

In [ ]:
def build_mod():
    in1 = Input((128,), name='x1')
    #print(in1.shape)
    x1 = fc_block(in1)
    #print(x1.shape)
    in2 = Input((1024,), name='x2')
    x2 = fc_block(in2)
    #x=x1
    #print(x2.shape)
    x = Concatenate(axis=-1)([x1, x2])
    #print(x.shape)
    # x = Dense(1024)(x)
    # x = BatchNormalization()(x)
    # x = LeakyReLU()(x)
    # x = Dropout(0.2)(x)
    x=fc_block(x)
    #print(x)
    out = Dense(2000, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=[in1,in2], outputs=out)
    #print(model.summary())
    opt = adam_v2.Adam(learning_rate=0.00001)
    #opt=gradient_descent_v2.SGD(learning_rate=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy')
    return model
def train(X_train,Y_train,X_val,Y_val):
    #if not os.path.exists('weights'): os.mkdir('weights')
    model = build_mod()
    callback = EarlyStopping(monitor='loss', patience=3,restore_best_weights=True)
    model.fit([X_train[0],X_train[1]],Y_train,epochs=100,batch_size=50,
                 validation_data = ([X_val[0],X_val[1]],Y_val),callbacks=[callback])
    model.save_weights('weights.h5')
    model.load_weights('weights.h5')
    y_prd = model.predict({'x1': X_val[0],'x2':X_val[1]}, verbose=False, batch_size=50)
    g = gap(y_prd, Y_val)
    hit=eval.calculate_hit_at_one(y_prd, y_val)
    print("Hit at one",hit)
    print("val GAP",g)
def predict(idx,X_test,Y_test,ylabels):
    model = build_mod()
    model.load_weights('weights.h5')
    ypd = model.predict({'x1': X_test[0],'x2':X_test[1]}, verbose=1, batch_size=10)
    g = gap(ypd, Y_test)
    hit=eval.calculate_hit_at_one(ypd, Y_test)
    print("Hit at one",hit)
    print("int",g)
    #del x1_val, x2_val
    with mp.Pool() as pool:
        out = pool.map(conv_pred, list(ypd))
        out1 = pool.map(another_pred, list(ypd))
    df = pd.DataFrame.from_dict({'VideoId': idx,"True labels":ylabels,"Pred labels":out1, 'Confidence': out})
    return df

In [ ]:
train(X_train,Y_train,X_val,Y_val)

Epoch 1/100
78/78 [==============================] - 69s 863ms/step - loss: 19.3420 - val_loss: 19.5680
Epoch 2/100
78/78 [==============================] - 78s 1s/step - loss: 14.6078 - val_loss: 14.4708
Epoch 3/100
78/78 [==============================] - 56s 714ms/step - loss: 12.4102 - val_loss: 13.0989
Epoch 4/100
78/78 [==============================] - 67s 864ms/step - loss: 11.5882 - val_loss: 12.6318
Epoch 5/100
78/78 [==============================] - 46s 591ms/step - loss: 11.2809 - val_loss: 12.6349
Epoch 6/100
78/78 [==============================] - 70s 895ms/step - loss: 11.3242 - val_loss: 12.9591
Epoch 7/100
78/78 [==============================] - 35s 447ms/step - loss: 11.5038 - val_loss: 13.4656
Epoch 8/100
78/78 [==============================] - 42s 542ms/step - loss: 11.8891 - val_loss: 14.0555
Hit at one 0.6089644513137558
val GAP 0.5416108063832797


In [ ]:
df_VidDense_all=predict(idx,X_test,Y_test,ylabels)
#Hit@1=0.58
#Gap=0.53

22/22 [==============================] - 1s 57ms/step
Hit at one 0.5844748858447488
int 0.5288556928281277


In [ ]:
df_VidDense_all.to_csv("data/Video_level_Dense_all.csv")